# Diet test by David

The goal of this problem is to minimize cost while ensuring that the diet meets certain requirements.

In [1]:
from pyomo.environ import *   #: Same as usual

In [25]:
from pyomo.opt import SolverFactory

In [2]:
m = ConcreteModel()

In [3]:
foods = ["QPwCheese", "MDwCheese", "LeBigMac", "FOFish", "McGChicken", "Fries", "McSausage", "LfMilk", "OJ"]
entree = ["QPwCheese", "MDwCheese", "LeBigMac", "FOFish", "McGChicken"]
side = ["Fries", "McSausage"]
drink = ["LfMilk", "OJ"]

In [4]:
m.food = Set(initialize=foods)

In [5]:
COST_DICT = dict.fromkeys(foods)

In [6]:
COST_DICT = {"QPwCheese": 1.84,
             "MDwCheese": 2.19, 
             "LeBigMac": 1.84, 
             "FOFish": 1.44, 
             "McGChicken": 2.29, 
             "Fries": 0.77, 
             "McSausage": 1.29, 
             "LfMilk": 0.60, 
             "OJ": 0.72}

In [7]:
m.cost = Param(m.food, initialize=COST_DICT, within=PositiveReals)

In [8]:
m.f_min = Param(m.food, within=NonNegativeReals, default=0.0)

In [9]:
MAX_FOOD_SUPPLY = 20.0

In [10]:
def f_max_validate (model, value, j):
    return m.f_max[j] > m.f_min[j]

In [11]:
m.f_max = Param(m.food, validate=f_max_validate, default=MAX_FOOD_SUPPLY)

In [12]:
NUTR = ["Cal", "Carbo", "Protein", "VitA", "VitC", "Calc", "Iron"]

In [13]:
m.nutr = Set(initialize=NUTR)

In [14]:
N_MIN = {"Cal": 2000, "Carbo": 350, "Protein": 55, "VitA": 100, "VitC": 100, "Calc": 100, "Iron": 100}

In [15]:
m.n_min = Param(m.nutr, initialize=N_MIN)

In [16]:
N_MAX = {"Cal": 0, "Carbo": 375, "Protein": 0, "VitA": 0, "VitC": 0, "Calc": 0, "Iron": 0}

In [17]:
m.n_max = Param(m.nutr, initialize=N_MAX)

In [18]:
AMT = {}
AMT[("QPwCheese","Cal")] = 510
AMT[("MDwCheese","Cal")] = 370
AMT[("LeBigMac","Cal")] = 500
AMT[("FOFish","Cal")] = 370
AMT[("McGChicken","Cal")] = 400
AMT[("Fries","Cal")] = 220
AMT[("McSausage","Cal")] = 345
AMT[("LfMilk","Cal")] = 110
AMT[("OJ","Cal")] = 80
AMT[("QPwCheese","Carbo")] = 34
AMT[("MDwCheese","Carbo")] = 35
AMT[("LeBigMac","Carbo")] = 42
AMT[("FOFish","Carbo")] = 38
AMT[("McGChicken","Carbo")] = 42
AMT[("Fries","Carbo")] = 26
AMT[("McSausage","Carbo")] = 27
AMT[("LfMilk","Carbo")] = 12
AMT[("OJ","Carbo")] = 20
AMT[("QPwCheese","Protein")] = 28
AMT[("MDwCheese","Protein")] = 24
AMT[("LeBigMac","Protein")] = 25
AMT[("FOFish","Protein")] = 14
AMT[("McGChicken","Protein")] = 31
AMT[("Fries","Protein")] = 3
AMT[("McSausage","Protein")] = 15
AMT[("LfMilk","Protein")] = 9
AMT[("OJ","Protein")] = 1
AMT[("QPwCheese","VitA")] = 15
AMT[("MDwCheese","VitA")] = 15
AMT[("LeBigMac","VitA")] = 6
AMT[("FOFish","VitA")] = 2
AMT[("McGChicken","VitA")] = 8
AMT[("Fries","VitA")] = 0
AMT[("McSausage","VitA")] = 4
AMT[("LfMilk","VitA")] = 10
AMT[("OJ","VitA")] = 2
AMT[("QPwCheese","VitC")] = 6
AMT[("MDwCheese","VitC")] = 10
AMT[("LeBigMac","VitC")] = 2
AMT[("FOFish","VitC")] = 0
AMT[("McGChicken","VitC")] = 15
AMT[("Fries","VitC")] = 15
AMT[("McSausage","VitC")] = 0
AMT[("LfMilk","VitC")] = 4
AMT[("OJ","VitC")] = 120
AMT[("QPwCheese","Calc")] = 30
AMT[("MDwCheese","Calc")] = 20
AMT[("LeBigMac","Calc")] = 25
AMT[("FOFish","Calc")] = 15
AMT[("McGChicken","Calc")] = 15
AMT[("Fries","Calc")] = 0
AMT[("McSausage","Calc")] = 20
AMT[("LfMilk","Calc")] = 30
AMT[("OJ","Calc")] = 2
AMT[("QPwCheese","Iron")] = 20
AMT[("MDwCheese","Iron")] = 20
AMT[("LeBigMac","Iron")] = 20
AMT[("FOFish","Iron")] = 10
AMT[("McGChicken","Iron")] = 8
AMT[("Fries","Iron")] = 2
AMT[("McSausage","Iron")] = 15
AMT[("LfMilk","Iron")] = 0
AMT[("OJ","Iron")] = 2

In [19]:
m.amt = Param(m.food, m.nutr, initialize=AMT)

In [20]:
def buy_bounds(mod, i):
    return (mod.f_min[i], mod.f_max[i])
m.buy = Var(m.food, bounds=buy_bounds, within=NonNegativeIntegers)

In [21]:
def total_cost_rule(mod):
    return sum(mod.cost[j] * mod.buy[j] for j in mod.food)
m.total_cost = Objective(rule=total_cost_rule, sense=minimize)

In [22]:
def entree_rule(mod):
    return sum(mod.buy[e] for e in entree) >= 1
m.entree = Constraint(rule=entree_rule)

In [23]:
def side_rule(mod):
    return sum(mod.buy[s] for s in side) >= 1
m.side = Constraint(rule=side_rule)

In [24]:
def drink_rule(mod):
    return sum(mod.buy[d] for d in drink) >=1
m.drink = Constraint(rule=drink_rule)

In [26]:
opt = SolverFactory('ipopt')  #: declare the solver

In [28]:
results = opt.solve(m, tee=True)

Ipopt 3.12.12: 
==> Warning: Treating 0 binary and 9 integer variables as continous.


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.12, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        9
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        9
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       